"Exploratorio"

In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

file_path = 'health_data.csv'
df = pd.read_csv(file_path)

display(df.head())

,Edad,Género,Estado civil,Altura,Peso,Índice de masa corporal,¿Fuma actualmente?,¿Fumó en el pasado?,¿Consume alcohol frecuentemente?,Nivel de actividad física,...,¿Sufre de problemas de visión?,¿Tiene problemas de audición?,¿Ha sufrido de fracturas óseas en el pasado?,Nivel de satisfacción con la vida,Enfermedad cardiovascular,Diabetes,Asma,Cáncer,Obesidad,Depresión/Ansiedad
0,76.596326,Otro,Soltero,153.681426,76.920289,29.612895,No,Sí,No,Moderado,...,No,No,No,Medio,0.674302,-0.171059,1.142946,0.293202,-0.130552,0.228336
1,79.795297,Otro,Casado,155.882307,66.743641,9.902543,No,No,No,Moderado,...,Sí,No,Sí,Bajo,-0.014915,0.101641,-0.059422,1.047785,0.216788,1.211533
2,90.603394,Otro,Casado,176.481841,124.818134,27.248719,Sí,No,Sí,Moderado,...,No,No,No,Medio,0.981927,0.054446,0.918964,0.138127,-0.030003,-0.205018
3,22.154276,Femenino,Viudo,158.681358,114.807668,27.634473,No,No,No,Moderado,...,No,Sí,No,Bajo,1.147131,0.256350,-0.159599,-0.260462,1.363624,0.291855
4,46.176676,Masculino,Casado,184.451263,60.217207,24.094841,No,Sí,No,Sedentario,...,No,No,No,Medio,1.067995,0.225792,0.165198,0.015367,0.960565,1.427700


In [4]:
missing_values = df.isnull().sum()
print("Missing values per column:")
print(missing_values[missing_values > 0])

duplicates = df.duplicated().sum()
print(f"\nNumber of duplicate rows: {duplicates}")

Missing values per column:
Series([], dtype: int64)

Number of duplicate rows: 0


# Transformaciones

In [10]:
import pandas as pd

# En este punto supongo que df ya está cargado:
# df = pd.read_csv("health_data.csv")

# Diccionario con las medias y desviaciones estándar que nos dieron
# para cada enfermedad. Lo uso para "deshacer" la estandarización.
stats = {
    "Enfermedad cardiovascular": {"mean": 0.303130, "std": 0.514926},
    "Diabetes": {"mean": 0.205536, "std": 0.450558},
    "Asma": {"mean": 0.150921, "std": 0.399708},
    "Cáncer": {"mean": 0.105589, "std": 0.342888},
    "Obesidad": {"mean": 0.248486, "std": 0.483748},
    "Depresión/Ansiedad": {"mean": 0.401899, "std": 0.545406}
}

# 1) Desestandarizo las columnas de enfermedades para volver a la escala original
for col, info in stats.items():
    # nueva columna *_real con el valor "deshecho" de la estandarización
    df[col + "_real"] = df[col] * info["std"] + info["mean"]

# 2) Paso cada enfermedad a binaria:
#    1 si la probabilidad es mayor o igual a 0.5, 0 si es menor
for col in stats:
    df[col + "_bin"] = (df[col + "_real"] >= 0.5).astype(int)

# Solo para chequear rápido cómo queda el dataframe
df.head()



,Edad,Género,Estado civil,Altura,Peso,Índice de masa corporal,¿Fuma actualmente?,¿Fumó en el pasado?,¿Consume alcohol frecuentemente?,Nivel de actividad física,...,Depresión/Ansiedad_real,Enfermedad cardiovascular_bin,Diabetes_bin,Asma_bin,Cáncer_bin,Obesidad_bin,Depresión/Ansiedad_bin,disease_combo,disease_combo_reduced,disease_class_final
0,76.596326,Otro,Soltero,153.681426,76.920289,29.612895,No,Sí,No,Moderado,...,0.526435,1,0,1,0,0,1,"(1, 0, 1, 0, 0, 1)","(1, 0, 1, 0, 0, 1)",0
1,79.795297,Otro,Casado,155.882307,66.743641,9.902543,No,No,No,Moderado,...,1.062676,0,0,0,0,0,1,"(0, 0, 0, 0, 0, 1)","(0, 0, 0, 0, 0, 1)",1
2,90.603394,Otro,Casado,176.481841,124.818134,27.248719,Sí,No,Sí,Moderado,...,0.290081,1,0,1,0,0,0,"(1, 0, 1, 0, 0, 0)","(1, 0, 1, 0, 0, 0)",2
3,22.154276,Femenino,Viudo,158.681358,114.807668,27.634473,No,No,No,Moderado,...,0.561079,1,0,0,0,1,1,"(1, 0, 0, 0, 1, 1)","(1, 0, 0, 0, 1, 1)",3
4,46.176676,Masculino,Casado,184.451263,60.217207,24.094841,No,Sí,No,Sedentario,...,1.180575,1,0,0,0,1,1,"(1, 0, 0, 0, 1, 1)","(1, 0, 0, 0, 1, 1)",3


In [11]:
# Me quedo con solo las columnas "reales" de las enfermedades
disease_cols = [
    "Enfermedad cardiovascular_real",
    "Diabetes_real",
    "Asma_real",
    "Cáncer_real",
    "Obesidad_real",
    "Depresión/Ansiedad_real"
]

df_diseases = df[disease_cols].copy()

# Guardo  en un CSV 
df_diseases.to_csv("diseases_only.csv", index=False, encoding="utf-8")


In [12]:
import pyreadr

pyreadr.write_rdata("diseases_only.RData", df_diseases, "diseases")

In [13]:
import numpy as np

# Columnas binarias de las 6 enfermedades (las que ya creaste)
disease_bin_cols = [
    "Enfermedad cardiovascular_bin",
    "Diabetes_bin",
    "Asma_bin",
    "Cáncer_bin",
    "Obesidad_bin",
    "Depresión/Ansiedad_bin"
]


df["disease_combo"] = df[disease_bin_cols].apply(lambda fila: tuple(fila.values), axis=1)


combo_counts = df["disease_combo"].value_counts()

min_count = 30   # si quieres menos clases, sube este número

combos_comunes = combo_counts[combo_counts >= min_count].index

def agrupar_combos(combo):
    if combo in combos_comunes:
        return combo
    else:
        return "OTRAS"

# Versión reducida de la combinación, con las raras agrupadas
df["disease_combo_reduced"] = df["disease_combo"].apply(agrupar_combos)

# Ahora paso estas etiquetas (tuplas + "OTRAS") a números 0,1,2,...
etiquetas_unicas = list(df["disease_combo_reduced"].unique())
label_to_id = {etq: i for i, etq in enumerate(etiquetas_unicas)}

df["disease_class_final"] = df["disease_combo_reduced"].map(label_to_id)

df[["disease_combo", "disease_combo_reduced", "disease_class_final"]].head()


,disease_combo,disease_combo_reduced,disease_class_final
0,"(1, 0, 1, 0, 0, 1)","(1, 0, 1, 0, 0, 1)",0
1,"(0, 0, 0, 0, 0, 1)","(0, 0, 0, 0, 0, 1)",1
2,"(1, 0, 1, 0, 0, 0)","(1, 0, 1, 0, 0, 0)",2
3,"(1, 0, 0, 0, 1, 1)","(1, 0, 0, 0, 1, 1)",3
4,"(1, 0, 0, 0, 1, 1)","(1, 0, 0, 0, 1, 1)",3


# Preparar X e y y seguir con la red neuronal

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, f1_score

disease_bin_cols = [
    "Enfermedad cardiovascular_bin",
    "Diabetes_bin",
    "Asma_bin",
    "Cáncer_bin",
    "Obesidad_bin",
    "Depresión/Ansiedad_bin"
]


target_col = "disease_class_final" 

# X: todas las columnas menos las que son puro target
cols_a_sacar = disease_bin_cols + [
    "disease_combo",
    "disease_class",
    "disease_class_reduced",
    "disease_class_final"
]

cols_a_sacar = [c for c in cols_a_sacar if c in df.columns]

X = df.drop(columns=cols_a_sacar)
y = df[target_col]

print("Shape X:", X.shape)
print("Clases distintas en y:", y.nunique())


Shape X: (20000, 63)
Clases distintas en y: 34


In [19]:
# Paso todas las columnas categóricas (tipo object) a dummies (one-hot)
cat_cols = X.select_dtypes(include=["object"]).columns
print("Columnas categóricas que voy a codificar:", list(cat_cols))

X = pd.get_dummies(X, columns=cat_cols, drop_first=False, dtype=int)

print("Shape X después de get_dummies:", X.shape)


Columnas categóricas que voy a codificar: ['Género', 'Estado civil', '¿Fuma actualmente?', '¿Fumó en el pasado?', '¿Consume alcohol frecuentemente?', 'Nivel de actividad física', '¿Tiene una dieta equilibrada?', '¿Consume frutas y verduras diariamente?', 'Frecuencia de consumo de comida rápida', '¿Duerme al menos 7 horas por noche?', '¿Experimenta estrés con frecuencia?', '¿Tiene antecedentes de hipertensión en la familia?', '¿Tiene antecedentes de diabetes en la familia?', '¿Tiene antecedentes de cáncer en la familia?', '¿Tiene antecedentes de enfermedades cardiovasculares en la familia?', '¿Tiene antecedentes de problemas de tiroides en la familia?', 'Frecuencia de ejercicio físico semanal', '¿Toma medicamentos regularmente?', 'Nivel de colesterol', 'Nivel de triglicéridos', 'Nivel de glucosa en sangre', 'Presión arterial', 'Consumo de sal en la dieta', '¿Tiene antecedentes de obesidad en la familia?', '¿Tiene antecedentes de asma?', '¿Padece de alguna alergia?', '¿Ha tenido infeccio

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, f1_score

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)


# Red Neuronal

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

num_features = X_train_scaled.shape[1]
num_classes  = y_train.nunique()

# Paso y a one-hot para usar softmax
y_train_cat = to_categorical(y_train, num_classes=num_classes)
y_test_cat  = to_categorical(y_test,  num_classes=num_classes)

model = Sequential()
model.add(Dense(64, activation="relu", input_shape=(num_features,)))
model.add(Dropout(0.3))
model.add(Dense(32, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation="softmax"))

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         9,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 34)             │         1,122 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,122 (51.26 KB)

 Trainable params: 13,122 (51.26 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
history = model.fit(
    X_train_scaled,
    y_train_cat,
    epochs=30,
    batch_size=128,
    validation_split=0.2,
    verbose=2
)


Epoch 1/30
100/100 - 1s - 14ms/step - accuracy: 0.2338 - loss: 2.8210 - val_accuracy: 0.6413 - val_loss: 1.5246
Epoch 2/30
100/100 - 0s - 1ms/step - accuracy: 0.6995 - loss: 1.1437 - val_accuracy: 0.9581 - val_loss: 0.3103
Epoch 3/30
100/100 - 0s - 1ms/step - accuracy: 0.8863 - loss: 0.4621 - val_accuracy: 0.9944 - val_loss: 0.0813
Epoch 4/30
100/100 - 0s - 1ms/step - accuracy: 0.9356 - loss: 0.2544 - val_accuracy: 0.9994 - val_loss: 0.0246
Epoch 5/30
100/100 - 0s - 1ms/step - accuracy: 0.9575 - loss: 0.1665 - val_accuracy: 1.0000 - val_loss: 0.0078
Epoch 6/30
100/100 - 0s - 1ms/step - accuracy: 0.9666 - loss: 0.1301 - val_accuracy: 1.0000 - val_loss: 0.0031
Epoch 7/30
100/100 - 0s - 1ms/step - accuracy: 0.9732 - loss: 0.1062 - val_accuracy: 1.0000 - val_loss: 0.0016
Epoch 8/30
100/100 - 0s - 1ms/step - accuracy: 0.9776 - loss: 0.0858 - val_accuracy: 1.0000 - val_loss: 9.1167e-04
Epoch 9/30
100/100 - 0s - 1ms/step - accuracy: 0.9801 - loss: 0.0725 - val_accuracy: 1.0000 - val_loss: 4.6

In [23]:
y_pred_proba = model.predict(X_test_scaled)
y_pred_class = y_pred_proba.argmax(axis=1)

print("F1 macro (Red Neuronal):",
      f1_score(y_test, y_pred_class, average="macro"))

print(classification_report(y_test, y_pred_class))


125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 407us/step
F1 macro (Red Neuronal): 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00       747
           2       1.00      1.00      1.00        44
           3       1.00      1.00      1.00       133
           4       1.00      1.00      1.00        63
           5       1.00      1.00      1.00       635
           6       1.00      1.00      1.00       329
           7       1.00      1.00      1.00        46
           8       1.00      1.00      1.00        19
           9       1.00      1.00      1.00        10
          10       1.00      1.00      1.00       190
          11       1.00      1.00      1.00       261
          12       1.00      1.00      1.00         6
          13       1.00      1.00      1.00        75
          14       1.00      1.00      1.00        32
          15       1.00      1.00      1.00        29
         

# Baggin con Red Neuronal


In [24]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def crear_modelo_nn(num_features, num_classes):
    """
    Armo una red chiquita estándar para el bagging.
    Así la puedo llamar muchas veces y siempre crea el mismo tipo de modelo.
    """
    m = Sequential()
    m.add(Dense(64, activation="relu", input_shape=(num_features,)))
    m.add(Dropout(0.3))
    m.add(Dense(32, activation="relu"))
    m.add(Dropout(0.3))
    m.add(Dense(num_classes, activation="softmax"))

    m.compile(
        optimizer="adam",
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )
    return m


In [25]:
import numpy as np
from sklearn.metrics import f1_score, classification_report

num_features = X_train_scaled.shape[1]
num_classes  = y_train.nunique()

n_models = 5  # cuántas redes quieres en el bagging (puedes subirlo si tu compu aguanta)
epocas_por_modelo = 20  # menos épocas por modelo para que no se haga eterno

modelos_bagging = []

for i in range(n_models):
    print(f"\nEntrenando modelo {i+1}/{n_models} del bagging...")

    # 1. Genero índices bootstrap: misma cantidad que el train, pero con reemplazo
    indices = np.random.choice(len(X_train_scaled), size=len(X_train_scaled), replace=True)
    X_boot = X_train_scaled[indices]
    y_boot = y_train.iloc[indices]

    # 2. One-hot para este bootstrap
    y_boot_cat = to_categorical(y_boot, num_classes=num_classes)

    # 3. Creo y entreno una red nueva
    m = crear_modelo_nn(num_features, num_classes)
    m.fit(
        X_boot,
        y_boot_cat,
        epochs=epocas_por_modelo,
        batch_size=128,
        verbose=0  # si quieres ver el entrenamiento, pon verbose=1
    )

    modelos_bagging.append(m)



Entrenando modelo 1/5 del bagging...


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Entrenando modelo 2/5 del bagging...

Entrenando modelo 3/5 del bagging...

Entrenando modelo 4/5 del bagging...

Entrenando modelo 5/5 del bagging...


In [26]:
# Predicciones de cada red
preds_proba = []

for i, m in enumerate(modelos_bagging):
    print(f"Prediciendo con modelo {i+1}/{n_models}...")
    p = m.predict(X_test_scaled, verbose=0)
    preds_proba.append(p)

# Promedio de probabilidades entre todos los modelos
preds_proba = np.array(preds_proba)          # shape: (n_models, n_muestras, num_classes)
mean_proba = preds_proba.mean(axis=0)        # shape: (n_muestras, num_classes)

y_pred_bagging_nn = mean_proba.argmax(axis=1)

print("F1 macro (bagging de redes):",
      f1_score(y_test, y_pred_bagging_nn, average="macro"))

print(classification_report(y_test, y_pred_bagging_nn))


Prediciendo con modelo 1/5...
Prediciendo con modelo 2/5...
Prediciendo con modelo 3/5...
Prediciendo con modelo 4/5...
Prediciendo con modelo 5/5...
F1 macro (bagging de redes): 1.0
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        50
           1       1.00      1.00      1.00       747
           2       1.00      1.00      1.00        44
           3       1.00      1.00      1.00       133
           4       1.00      1.00      1.00        63
           5       1.00      1.00      1.00       635
           6       1.00      1.00      1.00       329
           7       1.00      1.00      1.00        46
           8       1.00      1.00      1.00        19
           9       1.00      1.00      1.00        10
          10       1.00      1.00      1.00       190
          11       1.00      1.00      1.00       261
          12       1.00      1.00      1.00         6
          13       1.00      1.00      1.00        75
      